# Pre procesamiento

## Importar el archivo de tweets en CSV para el entrenamiento
Se importar el archivo y se eliminan las columnas que no se necesitan, se limpian datos innecesarios y con esto se genera un nuevo archivo con los tweets pre procesados

Basado en: https://towardsdatascience.com/another-twitter-sentiment-analysis-with-python-part-11-cnn-word2vec-41f5e28eda74

In [1]:
import pandas as pd  
import numpy as np
import matplotlib.pyplot as plt
import re

#Definir el nombre de las columnas del archivo
cols = ['id','XX','Polaridad','Texto']

#Importar el archivo en memoria 
#Se indica que no tiene encabezados
#Se usan los nombres definidos anteriormente los nombres de las columnas
df = pd.read_csv("tweets.csv",header=None, names=cols)

#Se botan las columnas que no se necesitan
df.drop(['id','XX'],axis=1,inplace=True)

#Se agrega columna de longitud original para validaciones luego del pre proccesamiento
df['LongitudOriginal'] = [len(str(t)) for t in df.Texto]

#Elimina las líneas que no tengan todos los valores definidos
df.dropna(inplace=True)
df.reset_index(drop=True,inplace=True)

#Imprimir las primeras líneas del archivo para validar una carga correcta
df.head()

,Polaridad,Texto,LongitudOriginal
0,NONE,Salgo de #VeoTV que día más largoooooo,38
1,NEU,@PauladeLasHeras No te libraras de ayudar meno...,63
2,P,@marodriguezb Gracias MAR,25
3,N+,Off pensando en el regalito Sinde la que se va...,128
4,P+,Conozco a alguien q es adicto al drama! Ja ja ...,65


In [2]:
#Imprimir el total de tweets por polaridad
df.Polaridad.value_counts()

P+      1654
NONE    1481
N       1334
P       1234
N+       846
NEU      669
Name: Polaridad, dtype: int64

In [5]:
#Patrones para hacer limpiezas
patronURL = r'https?:/?/?[A-Za-z0-9./]+'
patronWWW = r'www.[^ ]+'
patronNums = r'\d+'
patronElongaciones1Char = r'(.)\1+'
#Este último fue tomado de https://stackoverflow.com/questions/16884258/php-remove-duplicate-syllable-word
patronElongacionesSilabas = r'([b-df-hj-np-tv-xz][aeiouy](?:[a-z])?|[aeiouy][b-df-hj-np-tv-xz](?:[a-z])?)(\1){2,}'

def tweet_cleaner(text):    
    #Reemplaza URLs con una "palabra" única
    clean = re.sub(patronURL, 'xurlx', text)
    clean = re.sub(patronWWW, 'xurlx', clean)
    
    #Quita caracter identificar de UTF-8 BOM si existe
    try:
        clean = clean.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        clean = clean
    
    #Pasa todo a minúsculas
    clean = clean.lower()        
    #Reemplaza números con una "´palabra" única
    clean = re.sub(patronNums, "xnumx", clean)    
    #Reduce las elongaciones de un mismo caracter a solo 2 repeticiones
    clean = re.sub(patronElongaciones1Char, r'\1\1', clean) 
    #Reduce las elongaciones de una misma silaba a solo 2 repeticiones
    clean = re.sub(patronElongacionesSilabas, r'\1\1', clean) 
    
    return clean

#Prueba el pre procesamiento en un grupo pequeño de registros
testing = df.Texto[198:200]
test_result = []
for t in testing:
    print(t)
    test_result.append(tweet_cleaner(t))
print(test_result[0])
print(test_result[1])

Gracias a ti Paco RT @pacorengel #YNTERVIUMARIVI @mariviromero Muchísimas gracias Un beso La cuelgo en un ratito en http:tcoyK7PeIva
Aunque en menor medida también lo son muchas que no son madres ;) @ejoana Supermujeres Gran post de @virginiapalonso http:tcoSUTdmagF
gracias a ti paco rt @pacorengel #ynterviumarivi @mariviromero muchísimas gracias un beso la cuelgo en un ratito en xurlx
aunque en menor medida también lo son muchas que no son madres ;) @ejoana supermujeres gran post de @virginiapalonso xurlx


In [7]:
print ("Pre procesando tweets...\n")
clean_tweet_texts = []
for i in range(df.Texto.count()):
    if( (i+1)%1000 == 0 ):
        print("%d de %d tweets procesados" % ( i+1, df.Texto.count() ))
    clean_tweet_texts.append(tweet_cleaner(str(df['Texto'][i])))
print ("Pre procesamiento completado")

Pre procesando tweets...

1000 de 7218 tweets procesados
2000 de 7218 tweets procesados
3000 de 7218 tweets procesados
4000 de 7218 tweets procesados
5000 de 7218 tweets procesados
6000 de 7218 tweets procesados
7000 de 7218 tweets procesados
Pre procesamiento completado


In [8]:
#Generar dataframe con tweets pre procesados
clean_df = pd.DataFrame(clean_tweet_texts,columns=['Texto'])
clean_df['Polaridad'] = df.Polaridad
clean_df.head()

,Texto,Polaridad
0,salgo de #veotv que día más largoo,NONE
1,@pauladelasheras no te libraras de ayudar meno...,NEU
2,@marodriguezb gracias mar,P
3,off pensando en el regalito sinde la que se va...,N+
4,conozco a alguien q es adicto al drama! ja ja ...,P+


In [9]:
#Verificar si luego del pre procesamiento quedaron registros NULL 
#Hay que revisarlos contra los tweets originales y verificar si hay algún error
clean_df[clean_df.isnull().any(axis=1)]

,Texto,Polaridad


In [10]:
#Luego de revisar los registros NULL, se detectó que el único caso era porque no tenía comentario
#Se van a desechar los registros NULL
clean_df.dropna(inplace=True)
clean_df.reset_index(drop=True,inplace=True)
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7218 entries, 0 to 7217
Data columns (total 2 columns):
Texto        7218 non-null object
Polaridad    7218 non-null object
dtypes: object(2)
memory usage: 112.9+ KB


In [11]:
#Guarda los tweets a un archivo con encoding UTF-8
csv = 'tweets_limpios.csv'
clean_df.to_csv(csv,encoding='utf-8')
#Lo carga de nuevo para corroborar que esté bien
my_df = pd.read_csv(csv,index_col=0)
my_df.head()

,Texto,Polaridad
0,salgo de #veotv que día más largoo,NONE
1,@pauladelasheras no te libraras de ayudar meno...,NEU
2,@marodriguezb gracias mar,P
3,off pensando en el regalito sinde la que se va...,N+
4,conozco a alguien q es adicto al drama! ja ja ...,P+


In [15]:
#Finalmente, guardar un archivo con solo los textos para realizar la vectorización
texto_tweets = []
for i in range(my_df.Texto.count()):
    texto_tweets.append(str(my_df['Texto'][i]))
with open('SoloTextos.txt', 'w', encoding='utf-8') as f:
    for line in texto_tweets:
        f.write(line + "\n")